In [1]:
import pandas as pd
import re
from ast import literal_eval
from tqdm import tqdm
from ftfy import fix_text
# from util import UnitConversion, mapping_list_values, perl_to_posix
from groupby_toolz import enrich_db, gcloud
pd.set_option('display.max_colwidth', None)

pd.options.display.max_rows = 500
from flashtext import KeywordProcessor
from groupby_toolz import enrich_db, gcloud
pd.set_option('display.max_colwidth', None)
import ast
import warnings

warnings.filterwarnings('ignore')

In [2]:
# parameters
customer_id = '134'
formatted_attribute = 'micro_v_belt_overall_length'
attribute = formatted_attribute.lower().replace(' ','_').replace('-','_')

buckets = """V-Belt"""

regex_pattern = r"(?i)([^\.]\d* in\w*\s?OAL)|(\d*\.\d* in\w*\s?OAL)|(\d*\-\d*\/\d* in\w*\s?OAL)|(\d*\.\d in\w*\s?OAL)|([^\.]\d* in\w*\s?Overall Length)|(\d*\.\d* in\w*\s?Overall Length)|(\d*\-\d*\/\d* in\w*\s?Overall Length)|(\d*\.\d in\w*\s?Overall Length)"
# regex_pattern = r"(?i)(\b[8][1-9] in\w* Overall Length)|(\b[9][0-9] in\w* Overall Length)|(\b1[0][0] in\w* Overall Length)"


value='%n/a%'
customer_name='%motionindustriesus%'

params = {'customer_id': customer_id ,
          'attribute': attribute,
          'buckets': str(buckets.split('\t'))[1:-1],
          'value':value,
          'customer_name':customer_name
         }
curation_col = f'Q:{attribute}'

In [3]:
metric = ' Inches'

range_string = """
 0 - 20 Inches
 21 - 40 Inches
 41 - 60 Inches
 61 - 80 Inches
 81 - 100 Inches
 Greater than 100 Inches
 """

In [4]:
def query_from_file(file_name, params):
    with open(f'{file_name}', mode='r') as f:
        text = f.read()
        query = text.format(**params)
        return enrich_db(query)

In [5]:
df = query_from_file(file_name='./query/curated_products_VBOutDiamMM.sql', params=params)
df

attribute           value       customer_name  \
0     micro_v_belt_overall_length             n/a  motionindustriesus   
1     micro_v_belt_overall_length             n/a  motionindustriesus   
2     micro_v_belt_overall_length             n/a  motionindustriesus   
3     micro_v_belt_overall_length             n/a  motionindustriesus   
4     micro_v_belt_overall_length             n/a  motionindustriesus   
...                           ...             ...                 ...   
3239  micro_v_belt_overall_length  21 - 40 Inches  motionindustriesus   
3240  micro_v_belt_overall_length  21 - 40 Inches  motionindustriesus   
3241  micro_v_belt_overall_length  21 - 40 Inches  motionindustriesus   
3242  micro_v_belt_overall_length  21 - 40 Inches  motionindustriesus   
3243  micro_v_belt_overall_length   0 - 20 Inches  motionindustriesus   

     external_id  \
0       00456606   
1       00714493   
2       00720491   
3       00721316   
4       00721323   
...          ...   
3239    10538982   
3240    10538983   
3241    10539002   
3242    10539056   
3243    03220367   

                                                                                                                        long_desc  \
0     Micro-V® 917430114 Micro V-Belt - 14 Sections, M Size, 301.00 in Effective Length, 0.3750 in Rib Width, 5.2500 in Top Width   
1       Micro-V® 917102208 Micro V-Belt - 8 Sections, J Size, 22.00 in Effective Length, 0.0938 in Rib Width, 0.7500 in Top Width   
2      Micro-V® 917104024 Micro V-Belt - 24 Sections, J Size, 40.00 in Effective Length, 0.0938 in Rib Width, 2.2512 in Top Width   
3      Micro-V® 9171-01804 Micro V-Belt - 4 Sections, J Size, 18.00 in Effective Length, 0.0938 in Rib Width, 0.3750 in Top Width   
4       Micro-V® 917101904 Micro V-Belt - 4 Sections, J Size, 19.00 in Effective Length, 0.0938 in Rib Width, 0.3750 in Top Width   
...                                                                                                                           ...   
3239                                                              Micro-V Belt - K Section, 6 Ribs, 0.807 in Width, 119.31 in OAL   
3240                                                              Micro-V Belt - K Section, 6 Ribs, 0.807 in Width, 115.22 in OAL   
3241                                                               Micro-V Belt - K Section, 8 Ribs, 1.087 in Width, 41.36 in OAL   
3242                                                               Micro-V Belt - K Section, 9 Ribs, 1.227 in Width, 85.23 in OAL   
3243                                 FleetRunner® Micro-V Belt - K Section, 8 Ribs, 1.087 in Width, 13.12 in OAL, Standard Series   

     resolution  
0          bulk  
1          bulk  
2          bulk  
3          bulk  
4          bulk  
...         ...  
3239      rules  
3240      rules  
3241      rules  
3242      rules  
3243       bulk  

[3244 rows x 6 columns]

In [6]:
# Find a list of values to iterate
df_strategy = query_from_file(file_name='./asset/easy.sql', params=params) #create query
df_strategy
list_values=df_strategy['value'].to_list()
list_values

['0 - 20 Inches',
 '21 - 40 Inches',
 '41 - 60 Inches',
 '61 - 80 Inches',
 '81 - 100 Inches',
 'Greater than 100 Inches']

In [7]:
kwp=KeywordProcessor()
kwp.add_keywords_from_list(list_values)
# Extract Key Words
df['long_desc'].apply(lambda x: kwp.extract_keywords(x)) #

0       []
1       []
2       []
3       []
4       []
        ..
3239    []
3240    []
3241    []
3242    []
3243    []
Name: long_desc, Length: 3244, dtype: object

In [8]:
def re_extract(pattern, txt):
    matches = re.findall(pattern, txt)
    tmp_matches = []
    for match in matches:
        for tup in match:
            if tup != '':
                tmp_matches.append(tup)
    return list(set(tmp_matches))

In [9]:
df['matches'] = df['long_desc'].apply(lambda x: re_extract(regex_pattern, x))
x=df[df['matches'].astype(str)!='[]']

In [39]:
df[df['matches'].astype(str)=='[]']

attribute value       customer_name external_id  \
0     micro_v_belt_overall_length   n/a  motionindustriesus    00456606   
1     micro_v_belt_overall_length   n/a  motionindustriesus    00714493   
2     micro_v_belt_overall_length   n/a  motionindustriesus    00720491   
3     micro_v_belt_overall_length   n/a  motionindustriesus    00721316   
4     micro_v_belt_overall_length   n/a  motionindustriesus    00721323   
...                           ...   ...                 ...         ...   
2462  micro_v_belt_overall_length   n/a  motionindustriesus    08837833   
2463  micro_v_belt_overall_length   n/a  motionindustriesus    08837834   
2464  micro_v_belt_overall_length   n/a  motionindustriesus    08837835   
2465  micro_v_belt_overall_length   n/a  motionindustriesus    10538904   
2466  micro_v_belt_overall_length   n/a  motionindustriesus    10539054   

                                                                                                                        long_desc  \
0     Micro-V® 917430114 Micro V-Belt - 14 Sections, M Size, 301.00 in Effective Length, 0.3750 in Rib Width, 5.2500 in Top Width   
1       Micro-V® 917102208 Micro V-Belt - 8 Sections, J Size, 22.00 in Effective Length, 0.0938 in Rib Width, 0.7500 in Top Width   
2      Micro-V® 917104024 Micro V-Belt - 24 Sections, J Size, 40.00 in Effective Length, 0.0938 in Rib Width, 2.2512 in Top Width   
3      Micro-V® 9171-01804 Micro V-Belt - 4 Sections, J Size, 18.00 in Effective Length, 0.0938 in Rib Width, 0.3750 in Top Width   
4       Micro-V® 917101904 Micro V-Belt - 4 Sections, J Size, 19.00 in Effective Length, 0.0938 in Rib Width, 0.3750 in Top Width   
...                                                                                                                           ...   
2462                                                                                                                 Micro-V Belt   
2463                                                                                                                 Micro-V Belt   
2464                                                                                                                 Micro-V Belt   
2465                                                                                                               Micro-V Belt -   
2466                                                                                                               Micro-V Belt -   

     resolution matches  
0          bulk      []  
1          bulk      []  
2          bulk      []  
3          bulk      []  
4          bulk      []  
...         ...     ...  
2462       bulk      []  
2463       bulk      []  
2464       bulk      []  
2465       bulk      []  
2466       bulk      []  

[2467 rows x 7 columns]

In [10]:
x[x['value'].astype(str)!='n/a'][100:150]

attribute            value       customer_name  \
2567  micro_v_belt_overall_length  81 - 100 Inches  motionindustriesus   
2568  micro_v_belt_overall_length  81 - 100 Inches  motionindustriesus   
2569  micro_v_belt_overall_length  81 - 100 Inches  motionindustriesus   
2570  micro_v_belt_overall_length  81 - 100 Inches  motionindustriesus   
2571  micro_v_belt_overall_length  81 - 100 Inches  motionindustriesus   
2572  micro_v_belt_overall_length  81 - 100 Inches  motionindustriesus   
2573  micro_v_belt_overall_length  81 - 100 Inches  motionindustriesus   
2574  micro_v_belt_overall_length  81 - 100 Inches  motionindustriesus   
2575  micro_v_belt_overall_length  81 - 100 Inches  motionindustriesus   
2576  micro_v_belt_overall_length  81 - 100 Inches  motionindustriesus   
2577  micro_v_belt_overall_length  81 - 100 Inches  motionindustriesus   
2578  micro_v_belt_overall_length  81 - 100 Inches  motionindustriesus   
2579  micro_v_belt_overall_length  81 - 100 Inches  motionindustriesus   
2580  micro_v_belt_overall_length  81 - 100 Inches  motionindustriesus   
2581  micro_v_belt_overall_length  81 - 100 Inches  motionindustriesus   
2582  micro_v_belt_overall_length  81 - 100 Inches  motionindustriesus   
2583  micro_v_belt_overall_length  81 - 100 Inches  motionindustriesus   
2584  micro_v_belt_overall_length  81 - 100 Inches  motionindustriesus   
2585  micro_v_belt_overall_length  81 - 100 Inches  motionindustriesus   
2586  micro_v_belt_overall_length  81 - 100 Inches  motionindustriesus   
2587  micro_v_belt_overall_length  81 - 100 Inches  motionindustriesus   
2588  micro_v_belt_overall_length  81 - 100 Inches  motionindustriesus   
2589  micro_v_belt_overall_length  81 - 100 Inches  motionindustriesus   
2590  micro_v_belt_overall_length  81 - 100 Inches  motionindustriesus   
2591  micro_v_belt_overall_length  81 - 100 Inches  motionindustriesus   
2592  micro_v_belt_overall_length  81 - 100 Inches  motionindustriesus   
2593  micro_v_belt_overall_length  81 - 100 Inches  motionindustriesus   
2594  micro_v_belt_overall_length  81 - 100 Inches  motionindustriesus   
2595  micro_v_belt_overall_length  81 - 100 Inches  motionindustriesus   
2596  micro_v_belt_overall_length  81 - 100 Inches  motionindustriesus   
2597  micro_v_belt_overall_length  81 - 100 Inches  motionindustriesus   
2598  micro_v_belt_overall_length  81 - 100 Inches  motionindustriesus   
2599  micro_v_belt_overall_length  81 - 100 Inches  motionindustriesus   
2600  micro_v_belt_overall_length  81 - 100 Inches  motionindustriesus   
2601  micro_v_belt_overall_length  81 - 100 Inches  motionindustriesus   
2602  micro_v_belt_overall_length  81 - 100 Inches  motionindustriesus   
2603  micro_v_belt_overall_length  81 - 100 Inches  motionindustriesus   
2604  micro_v_belt_overall_length  81 - 100 Inches  motionindustriesus   
2605  micro_v_belt_overall_length  81 - 100 Inches  motionindustriesus   
2606  micro_v_belt_overall_length  81 - 100 Inches  motionindustriesus   
2607  micro_v_belt_overall_length  81 - 100 Inches  motionindustriesus   
2608  micro_v_belt_overall_length  81 - 100 Inches  motionindustriesus   
2609  micro_v_belt_overall_length  81 - 100 Inches  motionindustriesus   
2610  micro_v_belt_overall_length  81 - 100 Inches  motionindustriesus   
2611  micro_v_belt_overall_length  81 - 100 Inches  motionindustriesus   
2612  micro_v_belt_overall_length  81 - 100 Inches  motionindustriesus   
2613  micro_v_belt_overall_length  81 - 100 Inches  motionindustriesus   
2614  micro_v_belt_overall_length  81 - 100 Inches  motionindustriesus   
2615  micro_v_belt_overall_length  81 - 100 Inches  motionindustriesus   
2616  micro_v_belt_overall_length  81 - 100 Inches  motionindustriesus   

     external_id  \
2567    03220385   
2568    03220386   
2569    03619482   
2570    03939329   
2571    04244021   
2572    04426538   
2573    04426541   
2574    04426543   
2575    04426544   
2576    04426546   
2577    04426573   
2578    0

In [11]:
df[df['external_id'].astype(str)=='00739441']

attribute           value       customer_name  \
2633  micro_v_belt_overall_length  61 - 80 Inches  motionindustriesus   

     external_id  \
2633    00739441   

                                                                                         long_desc  \
2633  FleetRunner® Micro-V Belt - K Section, 4 Ribs, 0.560 in Width, 42.62 in OAL, Standard Series   

     resolution         matches  
2633      rules  [42.62 in OAL]

In [12]:
x['matches'].explode().value_counts()[50:100]

77.81 in OAL     2
59.96 in OAL     2
48.29 in OAL     2
70.75 in OAL     2
33.25 in OAL     2
40.625 in OAL    2
50.625 in OAL    2
73.50 in OAL     2
55.694 in OAL    2
116.27 in OAL    2
110.79 in OAL    2
34.25 in OAL     2
67.25 in OAL     2
71.135 in OAL    2
68.56 in OAL     2
 24 in OAL       2
87.59 in OAL     2
103.67 in OAL    2
101.63 in OAL    2
64.62 in OAL     2
52.00 in OAL     2
125.66 in OAL    2
39.09 in OAL     2
58.00 in OAL     2
104.31 in OAL    2
41.49 in OAL     2
55.27 in OAL     2
92.12 in OAL     2
36.125 in OAL    2
56.27 in OAL     2
60.56 in OAL     2
79.47 in OAL     2
41.25 in OAL     2
72.63 in OAL     2
84.799 in OAL    2
69.133 in OAL    2
73.79 in OAL     2
34.00 in OAL     2
90.87 in OAL     2
91.27 in OAL     2
86.93 in OAL     2
98.64 in OAL     2
93.58 in OAL     2
104.97 in OAL    2
88.387 in OAL    2
44.62 in OAL     2
65.00 in OAL     2
81.608 in OAL    2
83.75 in OAL     2
92.125 in OAL    2
Name: matches, dtype: int64

In [13]:
def split_col(data):
    data['matches']=data['matches'].str.get(0)
    # data=data.dropna()
    data['matches']=data['matches'].apply(lambda x:x.lower())

In [14]:
split_col(x)

In [15]:
len(x['matches'].explode().value_counts())

566

In [16]:
# error

In [17]:
x['matches']=x['matches'].apply(lambda measure: re.sub(r"[a-z]|[A-z]|\s|\w*\.\w*\.|\'",'',str(measure)))
x

attribute                    value  \
2467  micro_v_belt_overall_length  Greater than 100 Inches   
2468  micro_v_belt_overall_length  Greater than 100 Inches   
2469  micro_v_belt_overall_length  Greater than 100 Inches   
2470  micro_v_belt_overall_length  Greater than 100 Inches   
2471  micro_v_belt_overall_length  Greater than 100 Inches   
...                           ...                      ...   
3239  micro_v_belt_overall_length           21 - 40 Inches   
3240  micro_v_belt_overall_length           21 - 40 Inches   
3241  micro_v_belt_overall_length           21 - 40 Inches   
3242  micro_v_belt_overall_length           21 - 40 Inches   
3243  micro_v_belt_overall_length            0 - 20 Inches   

           customer_name external_id  \
2467  motionindustriesus    01476101   
2468  motionindustriesus    01543180   
2469  motionindustriesus    01606191   
2470  motionindustriesus    02102070   
2471  motionindustriesus    02102072   
...                  ...         ...   
3239  motionindustriesus    10538982   
3240  motionindustriesus    10538983   
3241  motionindustriesus    10539002   
3242  motionindustriesus    10539056   
3243  motionindustriesus    03220367   

                                                                                          long_desc  \
2467  FleetRunner® Micro-V Belt - K Section, 6 Ribs, 0.807 in Width, 102.97 in OAL, Standard Series   
2468  FleetRunner® Micro-V Belt - K Section, 7 Ribs, 0.947 in Width, 101.11 in OAL, Standard Series   
2469  FleetRunner® Micro-V Belt - K Section, 6 Ribs, 0.807 in Width, 111.16 in OAL, Standard Series   
2470  FleetRunner® Micro-V Belt - K Section, 6 Ribs, 0.807 in Width, 101.11 in OAL, Standard Series   
2471  FleetRunner® Micro-V Belt - K Section, 7 Ribs, 0.947 in Width, 101.11 in OAL, Standard Series   
...                                                                                             ...   
3239                                Micro-V Belt - K Section, 6 Ribs, 0.807 in Width, 119.31 in OAL   
3240                                Micro-V Belt - K Section, 6 Ribs, 0.807 in Width, 115.22 in OAL   
3241                                 Micro-V Belt - K Section, 8 Ribs, 1.087 in Width, 41.36 in OAL   
3242                                 Micro-V Belt - K Section, 9 Ribs, 1.227 in Width, 85.23 in OAL   
3243   FleetRunner® Micro-V Belt - K Section, 8 Ribs, 1.087 in Width, 13.12 in OAL, Standard Series   

     resolution matches  
2467       bulk  102.97  
2468       bulk  101.11  
2469       bulk  111.16  
2470       bulk  101.11  
2471       bulk  101.11  
...         ...     ...  
3239      rules  119.31  
3240      rules  115.22  
3241      rules   41.36  
3242      rules   85.23  
3243       bulk   13.12  

[777 rows x 7 columns]

In [18]:
def remove_fractions(numbers):
    if numbers.find('-') > -1:
#         numbers = literal_eval(numbers)
        inte, fract = numbers.split('-')
        nom, den = fract.split('/')
        tmp = float(nom)/float(den)
        num = int(inte) + tmp
        return str(num)
    else:
        return numbers

In [19]:
x['matches']=x['matches'].apply(lambda x: remove_fractions(x))
x

attribute                    value  \
2467  micro_v_belt_overall_length  Greater than 100 Inches   
2468  micro_v_belt_overall_length  Greater than 100 Inches   
2469  micro_v_belt_overall_length  Greater than 100 Inches   
2470  micro_v_belt_overall_length  Greater than 100 Inches   
2471  micro_v_belt_overall_length  Greater than 100 Inches   
...                           ...                      ...   
3239  micro_v_belt_overall_length           21 - 40 Inches   
3240  micro_v_belt_overall_length           21 - 40 Inches   
3241  micro_v_belt_overall_length           21 - 40 Inches   
3242  micro_v_belt_overall_length           21 - 40 Inches   
3243  micro_v_belt_overall_length            0 - 20 Inches   

           customer_name external_id  \
2467  motionindustriesus    01476101   
2468  motionindustriesus    01543180   
2469  motionindustriesus    01606191   
2470  motionindustriesus    02102070   
2471  motionindustriesus    02102072   
...                  ...         ...   
3239  motionindustriesus    10538982   
3240  motionindustriesus    10538983   
3241  motionindustriesus    10539002   
3242  motionindustriesus    10539056   
3243  motionindustriesus    03220367   

                                                                                          long_desc  \
2467  FleetRunner® Micro-V Belt - K Section, 6 Ribs, 0.807 in Width, 102.97 in OAL, Standard Series   
2468  FleetRunner® Micro-V Belt - K Section, 7 Ribs, 0.947 in Width, 101.11 in OAL, Standard Series   
2469  FleetRunner® Micro-V Belt - K Section, 6 Ribs, 0.807 in Width, 111.16 in OAL, Standard Series   
2470  FleetRunner® Micro-V Belt - K Section, 6 Ribs, 0.807 in Width, 101.11 in OAL, Standard Series   
2471  FleetRunner® Micro-V Belt - K Section, 7 Ribs, 0.947 in Width, 101.11 in OAL, Standard Series   
...                                                                                             ...   
3239                                Micro-V Belt - K Section, 6 Ribs, 0.807 in Width, 119.31 in OAL   
3240                                Micro-V Belt - K Section, 6 Ribs, 0.807 in Width, 115.22 in OAL   
3241                                 Micro-V Belt - K Section, 8 Ribs, 1.087 in Width, 41.36 in OAL   
3242                                 Micro-V Belt - K Section, 9 Ribs, 1.227 in Width, 85.23 in OAL   
3243   FleetRunner® Micro-V Belt - K Section, 8 Ribs, 1.087 in Width, 13.12 in OAL, Standard Series   

     resolution matches  
2467       bulk  102.97  
2468       bulk  101.11  
2469       bulk  111.16  
2470       bulk  101.11  
2471       bulk  101.11  
...         ...     ...  
3239      rules  119.31  
3240      rules  115.22  
3241      rules   41.36  
3242      rules   85.23  
3243       bulk   13.12  

[777 rows x 7 columns]

In [20]:
x['matches']=x['matches'].astype(float)

In [21]:
x

attribute                    value  \
2467  micro_v_belt_overall_length  Greater than 100 Inches   
2468  micro_v_belt_overall_length  Greater than 100 Inches   
2469  micro_v_belt_overall_length  Greater than 100 Inches   
2470  micro_v_belt_overall_length  Greater than 100 Inches   
2471  micro_v_belt_overall_length  Greater than 100 Inches   
...                           ...                      ...   
3239  micro_v_belt_overall_length           21 - 40 Inches   
3240  micro_v_belt_overall_length           21 - 40 Inches   
3241  micro_v_belt_overall_length           21 - 40 Inches   
3242  micro_v_belt_overall_length           21 - 40 Inches   
3243  micro_v_belt_overall_length            0 - 20 Inches   

           customer_name external_id  \
2467  motionindustriesus    01476101   
2468  motionindustriesus    01543180   
2469  motionindustriesus    01606191   
2470  motionindustriesus    02102070   
2471  motionindustriesus    02102072   
...                  ...         ...   
3239  motionindustriesus    10538982   
3240  motionindustriesus    10538983   
3241  motionindustriesus    10539002   
3242  motionindustriesus    10539056   
3243  motionindustriesus    03220367   

                                                                                          long_desc  \
2467  FleetRunner® Micro-V Belt - K Section, 6 Ribs, 0.807 in Width, 102.97 in OAL, Standard Series   
2468  FleetRunner® Micro-V Belt - K Section, 7 Ribs, 0.947 in Width, 101.11 in OAL, Standard Series   
2469  FleetRunner® Micro-V Belt - K Section, 6 Ribs, 0.807 in Width, 111.16 in OAL, Standard Series   
2470  FleetRunner® Micro-V Belt - K Section, 6 Ribs, 0.807 in Width, 101.11 in OAL, Standard Series   
2471  FleetRunner® Micro-V Belt - K Section, 7 Ribs, 0.947 in Width, 101.11 in OAL, Standard Series   
...                                                                                             ...   
3239                                Micro-V Belt - K Section, 6 Ribs, 0.807 in Width, 119.31 in OAL   
3240                                Micro-V Belt - K Section, 6 Ribs, 0.807 in Width, 115.22 in OAL   
3241                                 Micro-V Belt - K Section, 8 Ribs, 1.087 in Width, 41.36 in OAL   
3242                                 Micro-V Belt - K Section, 9 Ribs, 1.227 in Width, 85.23 in OAL   
3243   FleetRunner® Micro-V Belt - K Section, 8 Ribs, 1.087 in Width, 13.12 in OAL, Standard Series   

     resolution  matches  
2467       bulk   102.97  
2468       bulk   101.11  
2469       bulk   111.16  
2470       bulk   101.11  
2471       bulk   101.11  
...         ...      ...  
3239      rules   119.31  
3240      rules   115.22  
3241      rules    41.36  
3242      rules    85.23  
3243       bulk    13.12  

[777 rows x 7 columns]

# matches

In [22]:
matches=x
matches

attribute                    value  \
2467  micro_v_belt_overall_length  Greater than 100 Inches   
2468  micro_v_belt_overall_length  Greater than 100 Inches   
2469  micro_v_belt_overall_length  Greater than 100 Inches   
2470  micro_v_belt_overall_length  Greater than 100 Inches   
2471  micro_v_belt_overall_length  Greater than 100 Inches   
...                           ...                      ...   
3239  micro_v_belt_overall_length           21 - 40 Inches   
3240  micro_v_belt_overall_length           21 - 40 Inches   
3241  micro_v_belt_overall_length           21 - 40 Inches   
3242  micro_v_belt_overall_length           21 - 40 Inches   
3243  micro_v_belt_overall_length            0 - 20 Inches   

           customer_name external_id  \
2467  motionindustriesus    01476101   
2468  motionindustriesus    01543180   
2469  motionindustriesus    01606191   
2470  motionindustriesus    02102070   
2471  motionindustriesus    02102072   
...                  ...         ...   
3239  motionindustriesus    10538982   
3240  motionindustriesus    10538983   
3241  motionindustriesus    10539002   
3242  motionindustriesus    10539056   
3243  motionindustriesus    03220367   

                                                                                          long_desc  \
2467  FleetRunner® Micro-V Belt - K Section, 6 Ribs, 0.807 in Width, 102.97 in OAL, Standard Series   
2468  FleetRunner® Micro-V Belt - K Section, 7 Ribs, 0.947 in Width, 101.11 in OAL, Standard Series   
2469  FleetRunner® Micro-V Belt - K Section, 6 Ribs, 0.807 in Width, 111.16 in OAL, Standard Series   
2470  FleetRunner® Micro-V Belt - K Section, 6 Ribs, 0.807 in Width, 101.11 in OAL, Standard Series   
2471  FleetRunner® Micro-V Belt - K Section, 7 Ribs, 0.947 in Width, 101.11 in OAL, Standard Series   
...                                                                                             ...   
3239                                Micro-V Belt - K Section, 6 Ribs, 0.807 in Width, 119.31 in OAL   
3240                                Micro-V Belt - K Section, 6 Ribs, 0.807 in Width, 115.22 in OAL   
3241                                 Micro-V Belt - K Section, 8 Ribs, 1.087 in Width, 41.36 in OAL   
3242                                 Micro-V Belt - K Section, 9 Ribs, 1.227 in Width, 85.23 in OAL   
3243   FleetRunner® Micro-V Belt - K Section, 8 Ribs, 1.087 in Width, 13.12 in OAL, Standard Series   

     resolution  matches  
2467       bulk   102.97  
2468       bulk   101.11  
2469       bulk   111.16  
2470       bulk   101.11  
2471       bulk   101.11  
...         ...      ...  
3239      rules   119.31  
3240      rules   115.22  
3241      rules    41.36  
3242      rules    85.23  
3243       bulk    13.12  

[777 rows x 7 columns]

In [23]:
matches['matches']=matches['matches'].astype(float)

In [24]:
# range function builder

string = range_string.replace(metric, '')
lst = string.split('\n')
builder_base = f'''
def range_app(num):
'''
for val in lst:
    if val.find(' - ') >-1:
        original_val = val
        lower, upper = val.split(' - ')
        inner_f = f'''
    if num >= {lower} and num <= {upper}:
        return "{original_val}{metric}"
        '''
        builder_base = builder_base + inner_f
    if val.find('Greater than ') >-1:
        original_val = val
        upper = val.replace('Greater than ','')
        inner_f = f'''
    if num >= {upper}:
        return "{original_val}{metric}"
        '''
        builder_base = builder_base + inner_f
final_else = '''
    else:
        return r"n/a"
'''
exec(builder_base + final_else)
print( builder_base + final_else)


def range_app(num):

    if num >=  0 and num <= 20:
        return " 0 - 20 Inches"
        
    if num >=  21 and num <= 40:
        return " 21 - 40 Inches"
        
    if num >=  41 and num <= 60:
        return " 41 - 60 Inches"
        
    if num >=  61 and num <= 80:
        return " 61 - 80 Inches"
        
    if num >=  81 and num <= 100:
        return " 81 - 100 Inches"
        
    if num >=  100:
        return " Greater than 100 Inches"
        
    else:
        return r"n/a"



In [25]:
matches['matches']=matches['matches'].astype(float)
matches[curation_col] = matches['matches'].apply(range_app)
matches

attribute                    value  \
2467  micro_v_belt_overall_length  Greater than 100 Inches   
2468  micro_v_belt_overall_length  Greater than 100 Inches   
2469  micro_v_belt_overall_length  Greater than 100 Inches   
2470  micro_v_belt_overall_length  Greater than 100 Inches   
2471  micro_v_belt_overall_length  Greater than 100 Inches   
...                           ...                      ...   
3239  micro_v_belt_overall_length           21 - 40 Inches   
3240  micro_v_belt_overall_length           21 - 40 Inches   
3241  micro_v_belt_overall_length           21 - 40 Inches   
3242  micro_v_belt_overall_length           21 - 40 Inches   
3243  micro_v_belt_overall_length            0 - 20 Inches   

           customer_name external_id  \
2467  motionindustriesus    01476101   
2468  motionindustriesus    01543180   
2469  motionindustriesus    01606191   
2470  motionindustriesus    02102070   
2471  motionindustriesus    02102072   
...                  ...         ...   
3239  motionindustriesus    10538982   
3240  motionindustriesus    10538983   
3241  motionindustriesus    10539002   
3242  motionindustriesus    10539056   
3243  motionindustriesus    03220367   

                                                                                          long_desc  \
2467  FleetRunner® Micro-V Belt - K Section, 6 Ribs, 0.807 in Width, 102.97 in OAL, Standard Series   
2468  FleetRunner® Micro-V Belt - K Section, 7 Ribs, 0.947 in Width, 101.11 in OAL, Standard Series   
2469  FleetRunner® Micro-V Belt - K Section, 6 Ribs, 0.807 in Width, 111.16 in OAL, Standard Series   
2470  FleetRunner® Micro-V Belt - K Section, 6 Ribs, 0.807 in Width, 101.11 in OAL, Standard Series   
2471  FleetRunner® Micro-V Belt - K Section, 7 Ribs, 0.947 in Width, 101.11 in OAL, Standard Series   
...                                                                                             ...   
3239                                Micro-V Belt - K Section, 6 Ribs, 0.807 in Width, 119.31 in OAL   
3240                                Micro-V Belt - K Section, 6 Ribs, 0.807 in Width, 115.22 in OAL   
3241                                 Micro-V Belt - K Section, 8 Ribs, 1.087 in Width, 41.36 in OAL   
3242                                 Micro-V Belt - K Section, 9 Ribs, 1.227 in Width, 85.23 in OAL   
3243   FleetRunner® Micro-V Belt - K Section, 8 Ribs, 1.087 in Width, 13.12 in OAL, Standard Series   

     resolution  matches Q:micro_v_belt_overall_length  
2467       bulk   102.97       Greater than 100 Inches  
2468       bulk   101.11       Greater than 100 Inches  
2469       bulk   111.16       Greater than 100 Inches  
2470       bulk   101.11       Greater than 100 Inches  
2471       bulk   101.11       Greater than 100 Inches  
...         ...      ...                           ...  
3239      rules   119.31       Greater than 100 Inches  
3240      rules   115.22       Greater than 100 Inches  
3241      rules    41.36                41 - 60 Inches  
3242      rules    85.23               81 - 100 Inches  
3243       bulk    13.12                 0 - 20 Inches  

[777 rows x 8 columns]

In [34]:
matches[0:50]

attribute                    value  \
2467  micro_v_belt_overall_length  Greater than 100 Inches   
2468  micro_v_belt_overall_length  Greater than 100 Inches   
2469  micro_v_belt_overall_length  Greater than 100 Inches   
2470  micro_v_belt_overall_length  Greater than 100 Inches   
2471  micro_v_belt_overall_length  Greater than 100 Inches   
2472  micro_v_belt_overall_length  Greater than 100 Inches   
2473  micro_v_belt_overall_length  Greater than 100 Inches   
2474  micro_v_belt_overall_length  Greater than 100 Inches   
2475  micro_v_belt_overall_length  Greater than 100 Inches   
2476  micro_v_belt_overall_length  Greater than 100 Inches   
2477  micro_v_belt_overall_length  Greater than 100 Inches   
2478  micro_v_belt_overall_length  Greater than 100 Inches   
2479  micro_v_belt_overall_length  Greater than 100 Inches   
2480  micro_v_belt_overall_length  Greater than 100 Inches   
2481  micro_v_belt_overall_length  Greater than 100 Inches   
2482  micro_v_belt_overall_length  Greater than 100 Inches   
2483  micro_v_belt_overall_length  Greater than 100 Inches   
2484  micro_v_belt_overall_length  Greater than 100 Inches   
2485  micro_v_belt_overall_length  Greater than 100 Inches   
2486  micro_v_belt_overall_length  Greater than 100 Inches   
2487  micro_v_belt_overall_length  Greater than 100 Inches   
2488  micro_v_belt_overall_length  Greater than 100 Inches   
2489  micro_v_belt_overall_length  Greater than 100 Inches   
2490  micro_v_belt_overall_length  Greater than 100 Inches   
2491  micro_v_belt_overall_length  Greater than 100 Inches   
2492  micro_v_belt_overall_length  Greater than 100 Inches   
2493  micro_v_belt_overall_length  Greater than 100 Inches   
2494  micro_v_belt_overall_length  Greater than 100 Inches   
2495  micro_v_belt_overall_length  Greater than 100 Inches   
2496  micro_v_belt_overall_length  Greater than 100 Inches   
2497  micro_v_belt_overall_length  Greater than 100 Inches   
2498  micro_v_belt_overall_length  Greater than 100 Inches   
2499  micro_v_belt_overall_length  Greater than 100 Inches   
2500  micro_v_belt_overall_length  Greater than 100 Inches   
2501  micro_v_belt_overall_length  Greater than 100 Inches   
2502  micro_v_belt_overall_length  Greater than 100 Inches   
2503  micro_v_belt_overall_length  Greater than 100 Inches   
2504  micro_v_belt_overall_length  Greater than 100 Inches   
2505  micro_v_belt_overall_length  Greater than 100 Inches   
2506  micro_v_belt_overall_length  Greater than 100 Inches   
2507  micro_v_belt_overall_length  Greater than 100 Inches   
2508  micro_v_belt_overall_length  Greater than 100 Inches   
2509  micro_v_belt_overall_length  Greater than 100 Inches   
2510  micro_v_belt_overall_length  Greater than 100 Inches   
2511  micro_v_belt_overall_length  Greater than 100 Inches   
2512  micro_v_belt_overall_length  Greater than 100 Inches   
2513  micro_v_belt_overall_length  Greater than 100 Inches   
2514  micro_v_belt_overall_length  Greater than 100 Inches   
2515  micro_v_belt_overall_length  Greater than 100 Inches   
2516  micro_v_belt_overall_length  Greater than 100 Inches   

           customer_name external_id  \
2467  motionindustriesus    01476101   
2468  motionindustriesus    01543180   
2469  motionindustriesus    01606191   
2470  motionindustriesus    02102070   
2471  motionindustriesus    02102072   
2472  motionindustriesus    02294011   
2473  motionindustriesus    02467910   
2474  motionindustriesus    03213695   
2475  motionindustriesus    03520055   
2476  motionindustriesus    03662765   
2477  motionindustriesus    04162554   
2478  motionindustriesus    04402630   
2479  motionindustriesus    04426548   
2480  motionindustriesus    04426552   
2481  motionindustriesus    04426577   
2482  motionindustriesus    04426578   
2483  motionindustriesus    04426579   
2484  motionindustriesus    05699181   
2485  motionindustriesus    06771298   
2486  motionindustriesus    07599972   
2487  motionindustrie

In [27]:
matches['Q:micro_v_belt_overall_length'].explode().value_counts()

 61 - 80 Inches             204
 41 - 60 Inches             186
 81 - 100 Inches            166
 21 - 40 Inches             104
 Greater than 100 Inches     91
n/a                          25
 0 - 20 Inches                1
Name: Q:micro_v_belt_overall_length, dtype: int64

# fixed_na

In [35]:
fixed_na=matches[matches['Q:micro_v_belt_overall_length'].astype(str)=='n/a']

In [36]:
fixed_na['matches']=fixed_na['matches'].astype(float).apply(lambda x: x-1) #have to add 1 because these were 100.90 and it could not classify it
# na

In [37]:
fixed_na[curation_col] = fixed_na['matches'].apply(range_app)
fixed_na

attribute           value       customer_name  \
2634  micro_v_belt_overall_length  61 - 80 Inches  motionindustriesus   
2725  micro_v_belt_overall_length  61 - 80 Inches  motionindustriesus   
2742  micro_v_belt_overall_length  61 - 80 Inches  motionindustriesus   
2783  micro_v_belt_overall_length  61 - 80 Inches  motionindustriesus   
2791  micro_v_belt_overall_length  61 - 80 Inches  motionindustriesus   
2800  micro_v_belt_overall_length  61 - 80 Inches  motionindustriesus   
2822  micro_v_belt_overall_length  61 - 80 Inches  motionindustriesus   
2853  micro_v_belt_overall_length  61 - 80 Inches  motionindustriesus   
2880  micro_v_belt_overall_length  41 - 60 Inches  motionindustriesus   
2892  micro_v_belt_overall_length  41 - 60 Inches  motionindustriesus   
2894  micro_v_belt_overall_length  41 - 60 Inches  motionindustriesus   
2895  micro_v_belt_overall_length  41 - 60 Inches  motionindustriesus   
2903  micro_v_belt_overall_length  41 - 60 Inches  motionindustriesus   
2920  micro_v_belt_overall_length  41 - 60 Inches  motionindustriesus   
2926  micro_v_belt_overall_length  41 - 60 Inches  motionindustriesus   
2974  micro_v_belt_overall_length  41 - 60 Inches  motionindustriesus   
2989  micro_v_belt_overall_length  41 - 60 Inches  motionindustriesus   
2997  micro_v_belt_overall_length  41 - 60 Inches  motionindustriesus   
3014  micro_v_belt_overall_length  41 - 60 Inches  motionindustriesus   
3045  micro_v_belt_overall_length  41 - 60 Inches  motionindustriesus   
3123  micro_v_belt_overall_length  21 - 40 Inches  motionindustriesus   
3182  micro_v_belt_overall_length  21 - 40 Inches  motionindustriesus   
3219  micro_v_belt_overall_length  21 - 40 Inches  motionindustriesus   
3220  micro_v_belt_overall_length  21 - 40 Inches  motionindustriesus   
3225  micro_v_belt_overall_length  21 - 40 Inches  motionindustriesus   

     external_id  \
2634    00740939   
2725    04032896   
2742    04426572   
2783    07613432   
2791    07697965   
2800    08204957   
2822    10538779   
2853    10538930   
2880    00731547   
2892    01514881   
2894    01543154   
2895    01543155   
2903    01606181   
2920    02077922   
2926    02170106   
2974    04402624   
2989    04426563   
2997    04543687   
3014    05788740   
3045    10538743   
3123    02889514   
3182    07542910   
3219    10538877   
3220    10538878   
3225    10538896   

                                                                                                                 long_desc  \
2634                          FleetRunner® Micro-V Belt - K Section, 5 Ribs, 0.667 in Width, 20.73 in OAL, Standard Series   
2725                          FleetRunner® Micro-V Belt - K Section, 8 Ribs, 1.087 in Width, 80.18 in OAL, Standard Series   
2742                      FleetRunner® Micro-V Belt - K Section, 8 Ribs, 1.087 in Width, 80.19 in OAL, Fleet Runner Series   
2783      FleetRunner® 85523500 Automotive Micro V-Belt - K Section, 8 Ribs, 1.087 in Width, 80.87 in OAL, Standard Series   
2791      FleetRunner® 85522633 Automotive Micro V-Belt - K Section, 6 Ribs, 0.807 in Width, 80.07 in OAL, Standard Series   
2800  FleetRunner® 85550309 Automotive Micro V-Belt - K Section, 8 Ribs, 1.087 in Width, 80.75 in OAL, Fleet Runner Series   
2822                                                       Micro-V Belt - K Section, 8 Ribs, 1.087 in Width, 80.187 in OAL   
2853                                                       Micro-V Belt - K Section, 6 Ribs, 0.807 in Width, 80.326 in OAL   
2880                          FleetRunner® Micro-V Belt - K Section, 6 Ribs, 0.807 in Width, 60.71 in OAL, Standard Series   
2892                          FleetRunner® Micro-V Belt - K Section, 8 Ribs, 1.087 in Width, 60.56 in OAL, Standard Series   
2894                          FleetRunner® Micro-V Belt - K Section, 5 Ribs, 0.667 in Width, 40.62 in OAL, Standard Series   
2895                          FleetRunner® Micro-V Belt - K Section, 6 Ribs, 0.807 in Width,

In [38]:
fixed_na['Q:micro_v_belt_overall_length'].explode().value_counts()

 21 - 40 Inches    10
 41 - 60 Inches     7
 61 - 80 Inches     7
 0 - 20 Inches      1
Name: Q:micro_v_belt_overall_length, dtype: int64

# Wipe

In [66]:
wipe=df[['external_id']].astype(str)
wipe['Q:micro_v_belt_overall_length']=''
wipe

external_id Q:micro_v_belt_overall_length
0       00456606                              
1       00714493                              
2       00720491                              
3       00721316                              
4       00721323                              
...          ...                           ...
3239    10538982                              
3240    10538983                              
3241    10539002                              
3242    10539056                              
3243    03220367                              

[3244 rows x 2 columns]

# Null Values

In [62]:
na_external_id=df[df['matches'].astype(str)=='[]']
na_external_id['external_id']=na_external_id['external_id'].astype(str)

In [98]:
na_external_id

attribute value       customer_name external_id  \
0     micro_v_belt_overall_length   n/a  motionindustriesus    00456606   
1     micro_v_belt_overall_length   n/a  motionindustriesus    00714493   
2     micro_v_belt_overall_length   n/a  motionindustriesus    00720491   
3     micro_v_belt_overall_length   n/a  motionindustriesus    00721316   
4     micro_v_belt_overall_length   n/a  motionindustriesus    00721323   
...                           ...   ...                 ...         ...   
2462  micro_v_belt_overall_length   n/a  motionindustriesus    08837833   
2463  micro_v_belt_overall_length   n/a  motionindustriesus    08837834   
2464  micro_v_belt_overall_length   n/a  motionindustriesus    08837835   
2465  micro_v_belt_overall_length   n/a  motionindustriesus    10538904   
2466  micro_v_belt_overall_length   n/a  motionindustriesus    10539054   

                                                                                                                        long_desc  \
0     Micro-V® 917430114 Micro V-Belt - 14 Sections, M Size, 301.00 in Effective Length, 0.3750 in Rib Width, 5.2500 in Top Width   
1       Micro-V® 917102208 Micro V-Belt - 8 Sections, J Size, 22.00 in Effective Length, 0.0938 in Rib Width, 0.7500 in Top Width   
2      Micro-V® 917104024 Micro V-Belt - 24 Sections, J Size, 40.00 in Effective Length, 0.0938 in Rib Width, 2.2512 in Top Width   
3      Micro-V® 9171-01804 Micro V-Belt - 4 Sections, J Size, 18.00 in Effective Length, 0.0938 in Rib Width, 0.3750 in Top Width   
4       Micro-V® 917101904 Micro V-Belt - 4 Sections, J Size, 19.00 in Effective Length, 0.0938 in Rib Width, 0.3750 in Top Width   
...                                                                                                                           ...   
2462                                                                                                                 Micro-V Belt   
2463                                                                                                                 Micro-V Belt   
2464                                                                                                                 Micro-V Belt   
2465                                                                                                               Micro-V Belt -   
2466                                                                                                               Micro-V Belt -   

     resolution matches  
0          bulk      []  
1          bulk      []  
2          bulk      []  
3          bulk      []  
4          bulk      []  
...         ...     ...  
2462       bulk      []  
2463       bulk      []  
2464       bulk      []  
2465       bulk      []  
2466       bulk      []  

[2467 rows x 7 columns]

In [ ]:
error

In [65]:
gc = gcloud.Gsheets()

In [67]:
gc.save_to_gsheet('https://docs.google.com/spreadsheets/d/1A3dXssoEuvgun6YaO5jxM8P6_zZQ9KByEIZlhovF2gU/edit#gid=0','ul',wipe)

succesfully saved to worksheet:`ul` 
 url = `https://docs.google.com/spreadsheets/d/1A3dXssoEuvgun6YaO5jxM8P6_zZQ9KByEIZlhovF2gU/edit#gid=0`


# send to the folder for upload

In [69]:
def looks_good(customer, attribute, df, matches):
    drive_path = f'G:/Shared drives/GroupBy Public/Customer Success/.Enrich/Platform Upload Trail/{customer}/_Ready For Upload'
#     non_matches = df[df['matches'].astype(str) == '[]']
#     non_matches[curation_col] = r'n/a'
#     non_matches.to_csv(f'{drive_path}/Motion - {attribute} - na upload.csv',index=False)
    matches.to_csv(f'{drive_path}/Motion - {attribute} - match upload_Shawn.csv',index=False)

In [70]:
looks_good('Motion Industries', attribute, df, wipe)